## Setup

In [3]:
# Get raw advent-of-code data
from aocd.models import Puzzle

puzzle = Puzzle(year=2015, day=13)
input_data = puzzle.input_data
example = puzzle.examples[0]

In [4]:
# Import performance checking utility
import sys
from pathlib import Path

sys.path.append(str(Path.cwd().parent))

from common.utils.perf_check import check_example, time_solution

## Part a

### Combinatorial Approach
Another classic combinatorial / graph traversal problem. Let's start with a basic iterative approach that we also used on day 9.

In [5]:
# Imports
from collections import defaultdict
from itertools import permutations

In [6]:
# Functions
def solve_a_combinatorial(input_data: str) -> int:
    """Find the optimal seating arrangement to maximize total happiness using basic combinatorial iteration."""
    happiness = defaultdict(lambda: defaultdict(int))

    # Construct happiness map: {person_a: {person_b: happiness, ...}, ...}
    for line in input_data.splitlines():
        p_a, _, mode, hap_str, *_, p_b = line.split()

        hap = int(hap_str) * (-1 if mode == "lose" else 1)
        happiness[p_a][p_b[:-1]] += hap
        happiness[p_b[:-1]][p_a] += hap

    people = list(happiness.keys())

    return max(
        sum(happiness[perm[i]][perm[i + 1]] for i in range(len(perm) - 1)) + happiness[perm[-1]][perm[0]]
        for perm in permutations(people)
    )

In [17]:
# Correctness check
check_example(solve_a_combinatorial, example, "a")

solve_a_combinatorial found answer 330, which is the correct solution for part A!


True

In [18]:
# Performance check
time_a_combinatorial = time_solution(solve_a_combinatorial, input_data)

solve_a_combinatorial takes 30.73 ms


### Recursive approach
Let's also try a recursive approach based on day 9's solution.

In [7]:
# Imports
from functools import lru_cache

In [11]:
# Functions
def parse_to_matrix(input_data: str) -> list[list[int]]:
    """Parse input data into a happiness matrix and number of people."""
    lines = [line.split() for line in input_data.splitlines()]

    # Collect unique people and map to indices to allow bitmask operations for speed
    people = {p[0] for p in lines} | {p[-1][:-1] for p in lines}
    people_map = {name: i for i, name in enumerate(people)}
    n = len(people)

    # Initialize matrix with 0s
    matrix = [[0] * n for _ in range(n)]

    for p_a, _, mode, hap_str, *_, p_b in lines:
        hap_val = int(hap_str) * (-1 if mode == "lose" else 1)
        i, j = people_map[p_a], people_map[p_b[:-1]]
        matrix[i][j] += hap_val
        matrix[j][i] += hap_val

    return matrix


def solve_a_recursive(input_data: str | list[list[int]]) -> int:
    """Find the seating plan with maximum happiness by recursively checking all paths as bitmasks."""
    matrix = parse_to_matrix(input_data) if isinstance(input_data, str) else input_data
    n = len(matrix)

    # Bitmask when all people have been seated
    all_seated = (1 << n) - 1

    @lru_cache(5_000)
    def find_total_happiness(start: int, current_guest: int, seated_bitmask: int) -> int | float:
        # If mask is full, we've seated everyone
        if seated_bitmask == all_seated:
            # Close the loop by returning happiness from current guest to starting guest
            return matrix[current_guest][start]

        max_happiness = float("-inf")

        for nxt in range(n):
            if not (seated_bitmask & (1 << nxt)):
                # Cost = happiness from next guest + recursive happiness from next guests
                cost = matrix[current_guest][nxt] + find_total_happiness(start, nxt, seated_bitmask | (1 << nxt))

                max_happiness = max(max_happiness, cost)

        return max_happiness

    # Start from every person and pick the minimum
    if (result := max(find_total_happiness(i, i, 1 << i) for i in range(n))) == float("-inf"):
        msg = "No valid path found"
        raise ValueError(msg)

    return int(result)

In [32]:
# Correctness check
check_example(solve_a_recursive, example, "a")

solve_a_recursive found answer 330, which is the correct solution for part A!


True

In [39]:
# Performance check
time_a_recursive = time_solution(solve_a_recursive, input_data)
print(f" This is {time_a_combinatorial / time_a_recursive:.1f} times faster than the combinatorial approach.")

solve_a_recursive takes 2.85 ms
 This is 10.8 times faster than the combinatorial approach.


In [ ]:
# Submit answer
puzzle.answer_a = solve_a_recursive(input_data)

That's the right answer!  You are one gold star closer to powering the weather machine. [Continue to Part Two]


## Part b
The algorithm remains the same, we just need to add ourselves to the list of people with 0 happiness impact on anyone else.

I will just use the recursive approach here, and inject myself into the matrix with 0 impact on anyone else. I've adapted solve_a to be able to take in either raw input data string or a pre-calculated matrix.

In [9]:
# Functions
def solve_b(input_data: str) -> int:
    """Find the optimal seating arrangement including yourself with 0 happiness impact on anyone else."""
    # Parse original data to matrix
    matrix = parse_to_matrix(input_data)

    # Add ourselves as an additional person with 0 impact on anyone else
    for row in matrix:
        row.append(0)
    matrix.append([0] * (len(matrix) + 1))

    return solve_a_recursive(matrix)

In [13]:
# Performance check
time_b_pure_python = time_solution(solve_b, input_data)

solve_b takes 8.69 ms


In [14]:
# Submit answer
puzzle.answer_b = solve_b(input_data)

That's the right answer!  You are one gold star closer to powering the weather machine.You have completed Day 13! You can [Shareon
  Bluesky
Twitter
Mastodon] this victory or [Return to Your Advent Calendar].
